In [28]:
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense , Input , LSTM , Embedding, Dropout , Activation, GRU, Flatten,Conv2D,Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model, Sequential
from keras.layers import Convolution1D
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.utils import to_categorical
import numpy as np

In [3]:
df = pd.read_csv("C:/Users/Harvey/Desktop/Yelp_data_set/restuarant_review_raw_5_labels.csv",engine = 'python')

In [6]:
df.drop("language",axis = 1, inplace = True)

In [7]:
df.head()

,stars,text
0,5,Went in for a lunch. Steak sandwich was delici...
1,4,I'll be the first to admit that I was not exci...
2,3,Tracy dessert had a big name in Hong Kong and ...
3,1,This place has gone down hill. Clearly they h...
4,4,"Like walking back in time, every Saturday morn..."


In [8]:
df['text'][0]

"Went in for a lunch. Steak sandwich was delicious, and the Caesar salad had an absolutely delicious dressing, with a perfect amount of dressing, and distributed perfectly across each leaf. I know I'm going on about the salad ... But it was perfect.\r\n\r\nDrink prices were pretty good.\r\n\r\nThe Server, Dawn, was friendly and accommodating. Very happy with her.\r\n\r\nIn summation, a great pub experience. Would go again!"

In [10]:
df.columns  = ['stars','reviews']

In [13]:
df = df[['reviews','stars']]
df.head()

,reviews,stars
0,Went in for a lunch. Steak sandwich was delici...,5
1,I'll be the first to admit that I was not exci...,4
2,Tracy dessert had a big name in Hong Kong and ...,3
3,This place has gone down hill. Clearly they h...,1
4,"Like walking back in time, every Saturday morn...",4


In [14]:
# Remove \r \n 
df['reviews'] = df['reviews'].apply(lambda x : x.replace('\r\n',' ')) 

In [15]:
df['reviews'][0]

"Went in for a lunch. Steak sandwich was delicious, and the Caesar salad had an absolutely delicious dressing, with a perfect amount of dressing, and distributed perfectly across each leaf. I know I'm going on about the salad ... But it was perfect.  Drink prices were pretty good.  The Server, Dawn, was friendly and accommodating. Very happy with her.  In summation, a great pub experience. Would go again!"

In [16]:
import re
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

stop_words = set(stopwords.words("english")) 
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    text = re.sub(r'[^\w\s]','',text, re.UNICODE)
    text = text.lower()
    text = [lemmatizer.lemmatize(token,pos = "v") for token in text.split(" ")]
    text = [lemmatizer.lemmatize(token, "v") for token in text]
    text = [word for word in text if not word in stop_words]
    text = " ".join(text)
    return text

In [17]:
df['Processed_Reviews'] = df['reviews'].apply(lambda x: clean_text(x))

In [18]:
df.head()

,reviews,stars,Processed_Reviews
0,Went in for a lunch. Steak sandwich was delici...,5,go lunch steak sandwich delicious caesar salad...
1,I'll be the first to admit that I was not exci...,4,ill first admit excite go la tavolta food snob...
2,Tracy dessert had a big name in Hong Kong and ...,3,tracy dessert big name hong kong one first mar...
3,This place has gone down hill. Clearly they h...,1,place go hill clearly cut back staff food qua...
4,"Like walking back in time, every Saturday morn...",4,like walk back time every saturday morning sis...


In [19]:
df.drop('reviews',axis = 1 , inplace = True)
df.head()

,stars,Processed_Reviews
0,5,go lunch steak sandwich delicious caesar salad...
1,4,ill first admit excite go la tavolta food snob...
2,3,tracy dessert big name hong kong one first mar...
3,1,place go hill clearly cut back staff food qua...
4,4,like walk back time every saturday morning sis...


In [ ]:
# 5 label is the one after lemmatize and removing stop words
#df.to_csv("C:/Users/Harvey/Desktop/Yelp_data_set/restuarant_review_5_label.csv")

In [ ]:
#df = pd.read_csv("C:/Users/Harvey/Desktop/Yelp_data_set/restuarant_review_5_label.csv")
#df.drop("Unnamed: 0",axis = 1 , inplace = True)
#print(df.shape)
#df.head()

In [20]:
star1 = df[df['stars'] == 1]
print("1 star reviews shape" + str(star1.shape[0]))
star2 = df[df['stars'] == 2]
print("2 star reviews shape" + str(star2.shape[0]))
star3 = df[df['stars'] == 3]
print("3 star reviews shape" + str(star3.shape[0]))
star4 = df[df['stars'] == 4]
print("4 star reviews shape" + str(star4.shape[0]))
star5 = df[df['stars'] == 5]
print("5 star reviews shape" + str(star5.shape[0]))


1 star reviews shape67046
2 star reviews shape53939
3 star reviews shape77547
4 star reviews shape154883
5 star reviews shape237045


In [21]:
def remove_space_numbers(x):
    text = x.split(" ")
    text = [words for words in text if words.isdigit() == False and words != '']
    text = " ".join(text)
    return text

In [22]:
df['Processed_Reviews'] = df['Processed_Reviews'].apply(lambda x:remove_space_numbers(x))
df['Processed_Reviews'][0]
df.head()

,stars,Processed_Reviews
0,5,go lunch steak sandwich delicious caesar salad...
1,4,ill first admit excite go la tavolta food snob...
2,3,tracy dessert big name hong kong one first mar...
3,1,place go hill clearly cut back staff food qual...
4,4,like walk back time every saturday morning sis...


In [23]:
df.to_csv("C:/Users/Harvey/Desktop/Yelp_data_set/restuarant_review_5_label_unbalanced.csv",index = False)

In [24]:
df = pd.read_csv("C:/Users/Harvey/Desktop/Yelp_data_set/restuarant_review_5_label_unbalanced.csv")
df['Processed_Reviews'][0]

'go lunch steak sandwich delicious caesar salad absolutely delicious dress perfect amount dress distribute perfectly across leaf know im go salad perfect drink price pretty good server dawn friendly accommodate happy summation great pub experience would go'

In [25]:
df.head()

,stars,Processed_Reviews
0,5,go lunch steak sandwich delicious caesar salad...
1,4,ill first admit excite go la tavolta food snob...
2,3,tracy dessert big name hong kong one first mar...
3,1,place go hill clearly cut back staff food qual...
4,4,like walk back time every saturday morning sis...


In [32]:
train = df.sample(frac = 0.8,random_state = 200)
test = df.drop(train.index)

In [34]:
train.loc[:,'stars'] -= 1
print(np.unique(train['stars']))

[0 1 2 3 4]


In [35]:
train['Processed_Reviews'][1]

'ill first admit excite go la tavolta food snob group friends suggest go dinner look online menu nothing special seem overprice im also big order pasta go alas outnumber thank goodness order sea bass special die cook perfectly season perfectly perfect portion say enough good things dish server ask seem proud dish say doesnt chef incredible job hubby get crab tortellini also love hear mmmm good around table waiter super nice even give us free desserts last people restaurant service slow place pack jug wine large group good conversation didnt seem bother anyone order calamari fry zucchini appetizers leave mussels sea bass special, highly recommend it. chicken parm crab tortellini also good big. chicken romano bite bland. house salads teeny. make reservation still expect wait food. go large group people plan loud. go date unless fight feel like hear anything say. ask sit side room available.'